In [1]:
import numpy as np
import pandas as pd
import os,glob,math
import matplotlib.pyplot as plt
# import seaborn as sns
import matplotlib as mpl
from scipy import stats
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
plt.rc("font",family="Arial")


In [2]:
def fisher_dataframe(df):
    s1 = df.iloc[:,0]
    s2 = df.iloc[:,1]
    s1_sum = s1.sum()
    s2_sum = s2.sum()
    s3 = []
    for i,j in zip(s1,s2):
        group1 = [i,s1_sum-i]
        group2 = [j,s2_sum-j]
        t_statistic, p_value = stats.fisher_exact([group1, group2])
        s3.append(min(p_value*16*31,1))
    s4 = pd.Series(s3)
    s4.index = s1.index
    return s4

In [3]:
df= pd.read_excel("all_infor_for_stack_plot.xlsx",index_col=0)

In [7]:
dfFC = pd.read_csv("geno_base_pvalue.csv",index_col=0)

In [11]:
# FC based sort  Fisher p new 1 bar
for j in range(1,32):
    dfnew = pd.DataFrame(df.loc[df.cluster ==j].geno.value_counts())
    dfnew['cumulative'] = dfnew['geno'].cumsum()
    dfnew['percent'] = dfnew['geno']/dfnew.cumulative[-1]*100


    dfnew2 = pd.DataFrame(df.loc[df.cluster !=j].geno.value_counts())
    dfnew2['cumulative'] = dfnew2['geno'].cumsum()
    dfnew2['percent'] = dfnew2['geno']/dfnew2.cumulative[-1]*100

    dfno = pd.concat([dfnew.geno,dfnew2.geno],axis=1).fillna(0)
    sp = fisher_dataframe(dfno)

    dfplot_tmp = pd.concat([dfnew[["geno","percent"]],dfnew2[["geno","percent"]],sp],axis=1).fillna(0)
    dfplot_tmp.columns = ["In_value","In_percent","Out_value","Out_percent","p_value"]
    dfplot_tmp2 = dfplot_tmp.sort_values(by="In_percent",ascending=False)

    if str(j) not in dfFC.columns.tolist():
        my_list = dfplot_tmp2.index.tolist()
    else:   
        geno_tmp = dfFC[str(j)].loc[dfFC[str(j)]<=0.05].sort_values().index.tolist()
        my_list = geno_tmp+[x for x in dfplot_tmp2.index.tolist() if x not in geno_tmp]
    dfplot = dfplot_tmp2.loc[my_list,:].iloc[0:1,:]

    x = np.arange(len(dfplot))  # the label locations
    width = 0.35  # the width of the bars
    multiplier = 0
    # fig, ax = plt.subplots()
    tab10_colors = plt.cm.Pastel1.colors

    fig, ax = plt.subplots(figsize = (15,30))
    for percent, values,c,ec in zip(dfplot[["In_percent","Out_percent"]].iteritems(),dfplot[["In_value","Out_value"]].iteritems(),[tab10_colors[1],"none"],["none",(0.2,0.2,0.2)]):
        offset = width * multiplier
        rects = ax.bar(x + offset,np.round(percent[1].values, 1), width,color=c,edgecolor = ec,linewidth=7)


        multiplier += 1

    for i,s in enumerate(dfplot.iterrows()):
        # print(i)
        if (s[1]["p_value"]<0.05)&(s[1]["p_value"]>=0.01):
            max_y = max(s[1].iloc[[1,3]])+2
            ax.plot([i,i+0.4],[max_y,max_y],c="k",linewidth =10)
            ax.text(i+0.1+0.1,max_y,"*",fontsize = 100)
        elif (s[1]["p_value"]<0.01)&(s[1]["p_value"]>=0.001):
            max_y = max(s[1].iloc[[1,3]])+2
            ax.plot([i,i+0.4],[max_y,max_y],c="k",linewidth =10)
            ax.text(i+0.1+0.05,max_y,"**",fontsize = 100)
        elif s[1]["p_value"]<0.001:
            max_y = max(s[1].iloc[[1,3]])+2
            ax.plot([i,i+0.4],[max_y,max_y],c="k",linewidth =10)
            ax.text(i+0.1,max_y,"***",fontsize = 100)
    
        # Add some text for labels, title and custom x-axis tick labels, etc.
    # ax.plot([1,10],[20,],c="red")
    ax.set_ylabel('% of neurons',fontsize = 60)
    ax.yaxis.set_label_coords(-0.05, 0.5) 
    ax.set_title('Subtype #%d'%j,fontsize = 80)
    # 仅更改y轴刻度标签的字体大小
    ax.tick_params(axis='y', labelsize=40)
    ax.set_xticks(x + width, dfplot.index.tolist(),fontsize = 80)
    # ax.legend(loc='upper left', ncols=3)
    ax.set_ylim(0,math.ceil(dfplot[["In_percent","Out_percent"]].max().max())+4) 
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_linewidth(5)
    ax.spines["bottom"].set_linewidth(5)
    plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)
    # fig.set_figheight(6)
    plt.savefig("Subtype_%d.jpg"%j)
    # plt.savefig("Subtype_%d.pdf"%j,dpi=300)
    plt.close()
    

    